### IMPORTS

In [95]:
import numpy as np
import scipy as sp

$\textbf{QUESTION 1}$
$$
\textbf{Linear Program Objective Function:}\\
\text{min}\ 3x_1 + 2x_2
\\~\\
\textbf{Constraints:}\\
2x_1 + x_2 \ge 4\\
x_1 + x_2 \ge 3\\
x_1 + 2x_2 \le 10\\
x_1 \ge 0\\
x_2 \ge 0
\\~\\
\text{The feasible region is the shaded portion in the below graph.}
$$
<p align="center">
<img src="/home/leonardo_leads/Documents/SchoolDocs/ga_tech_masters/omscs_ml/fall2023/deterministic_optimization/lectures/week8Modules15and16/homework/images/Q1.png" width="600" height="600" />
</p>

$\textbf{QUESTION 2}$

$$
\textbf{Standard Form Linear Program:}\\
\textbf{Objective Function:}\\
\text{min}\ 3x_1 + 2x_2
\\~\\
\textbf{Constraints:}\\
2x_1 + x_2 - x_3 = 4\\
x_1 + x_2 - x_4 = 3\\
x_1 + 2x_2 +x_5 = 10\\
x_1, x_2, x_3, x_4, x_5 \ge 0
\\~\\
x = 
\begin{bmatrix}
x_1 & x_2 & x_3 & x_4 & x_5
\end{bmatrix}^T
\\~\\
c = 
\begin{bmatrix}
3 & 2 & 0 & 0 & 0
\end{bmatrix}^T
\\~\\
A =
\begin{bmatrix}
2 & 1 & -1 & 0 & 0\\
1 & 1 & 0 & -1 & 0\\
1 & 2 & 0 & 0 & 1
\end{bmatrix}
\\~\\
b = 
\begin{bmatrix}
4 & 3 & 10
\end{bmatrix}
$$

$\textbf{QUESTION 3}$
$$
\textbf{PART A}\\
$$

In [163]:
# c = np.array([3, 2, 0, 0, 0]).T
# A = np.matrix([[2, 1, -1, 0, 0],
#                [1, 1, 0, -1, 0],
#                [1, 2, 0, 0, 1],])
# b = np.array([4, 3, 10]).T

c = np.array([-2, -3, 0, 0, 0]).T
A = np.matrix([[-1, 1, 1, 0, 0],
               [3, 2, 0, 1, 0],
               [2, 3, 0, 0, 1],])
b = np.array([10, 60, 60]).T

c_ind = [i for i in range(c.shape[0])]
A_col_ind = [i for i in range(A.shape[1])]
b_ind = [i for i in range(b.shape[0])]

bfs = np.array([A[:, [2, 3, 4]]])
x_n = [0, 0]

while True:

    b_inv = np.linalg.inv(bfs)
    x_b = b_inv @ b

    c_b = c[[2, 3, 4]]
    c_n = c[[0, 1]]

    if x_b.all():
        print(f"Current basic solution is a BFS: {x_b=}\n{x_n=}")
    else: print("Current basic solution is not a BFS")

    A1 = A[:, 0]
    A2 = A[:, 1]

    c_n_bar1 = c_n[0] - c_b.T @ b_inv @ A1
    c_n_bar2 = c_n[1] - c_b.T @ b_inv @ A2
    print(c_n_bar1)
    print(c_n_bar2)

    if c_n_bar1 < 0 and c_n_bar2 < 0: 
        print("Current BFS is not optimal!")
    else:
        print("Current BFS is optimal!")
        break

    d_b = -b_inv @ A2
    d_n = np.array([0, 1])

    if d_b.any() < 0:
        print("Do not have an unbounded optimal solution.")
    else: print("Do have unbounded optimal solution.")

    # MIN-RATIO TEST
    theta = np.min(x_b / -d_b)
    theta_b, theta_n = x_b / -d_b, [0, theta]

    bfs = np.array([A[:, [1, 3, 4]]])
    b_inv = np.linalg.inv(bfs)
    x_b = b_inv @ b

    break














Current basic solution is a BFS: x_b=array([[10., 60., 60.]])
x_n=[0, 0]
[[-2.]]
[[-3.]]
Current BFS is not optimal!
Do have unbounded optimal solution.


In [ ]:
# def Simplex(A, b, c):
#     '''Takes input vars, computs corresponding values,
#     then uses while loop to iterate until a basic optimal solution is reached.
#     RETURNS: basic_cost_coeff, basic_cost_index, non_basic_cost, nonbasic_cost_index, bHat, cnHat.
#     basic_cost_coeff, cbIndex is final basic variable values, and indices
#     non_basic_cost, cnIndex is final nonbasic variable values and indices
#     bHat is final solution values, 
#     cnHat is optimality condition'''
    
#     # sizes of basic and nonbasic vectors
#     basicSize = A.shape[0]
#     nonbasicSize = A.shape[1] - basicSize # n - m, number of variables
#     print(basicSize)
#     print(nonbasicSize)
        
#     # global index tracker of variables of basic and nonbasic variables (objective)
#     cindx = [i for i in range(0, len(c))]
#     print(cindx)
    
#     # basic variable coefficients
#     basic_cost_coeff = np.array(c[:nonbasicSize])
#     print(basic_cost_coeff)
        
#     # nonbasic variable coefficients
#     non_basic_cost = np.array(c[nonbasicSize:])
#     print(non_basic_cost)

#     start_AInd = np.array([0, 2, 3])
#     basis = A[:, start_AInd]
#     k = 1
 
#     # run core simplex method until reach the optimal solution
#     while True:
        
#         basic_cost_index = cindx[:nonbasicSize]
#         nonbasic_cost_index = cindx[nonbasicSize:]
    
#         # basis matrix
#         basis = A[:, basic_cost_index]
#         basic_var = np.linalg.inv(basis)
        
#         # nonbasic variable matrix
#         N = A[:, nonbasic_cost_index]

#         # bHat, the values of the basic variables
#         # recall that at the start the basic variables are the slack variables, and 
#         # have values equal to the vector b (as primary variables are set to 0 at the start)
#         bHat = basic_var @ b
#         yT = basic_cost_coeff @ basic_var
        
#         # use to check for optimality, determine variable to enter basis
#         cnHat = non_basic_cost - (yT @ N)
        
        
#         # find indx of minimum value of cnhat, this is the variable to enter the basis
#         cnMinIndx = np.argmin(cnHat)

#         # break out of loop, returning values if all values of cnhat are above 0
#         if all(i >= 0 for i in list([cnHat[0,0], cnHat[0,1]])):
#             # use basic_cost_index to get index values of variables in bHat, and the corresponding index
#             # values in bHat are the final solution values for each of the corresponding variables
#             # ie value 0 in dbIndx corresponds with first variable, so whatever the index for the 0 is
#             # is the index in bHat that has the solution value for that variable.
#             return f"{k=}\n{basic_cost_coeff=}\n{basic_cost_index=}\n{non_basic_cost=}\n{nonbasic_cost_index=}\n{bHat=}\n{cnHat=}"
    
#         # this is the index for the column of coeffs in A for the given variable
#         indx = cindx[cnMinIndx]

#         Ahat = basic_var @ A[:, indx]
        
#         # now we want to iterate through Ahat and bHat and pick the minimum ratios
#         # only take ratios of variables with Ahat_i values greater than 0
#         # pick smallest ratio to get variable that will become nonbasic.
#         ratios = []
#         for i in range(0, len(bHat)):
#             Aval = Ahat[i]
#             Bval = bHat[i]

#             # don't look at ratios with val less then or eqaul to 0, append to keep index
#             if(Aval <= 0):
#                 ratios.append(10000000)
#                 continue
#             ratios.append(Bval / Aval)

#         ratioMinIndx = np.argmin(ratios)

#         # switch basic and nonbasic variables using the indices.
#         non_basic_cost[cnMinIndx], basic_cost_coeff[ratioMinIndx] = basic_cost_coeff[ratioMinIndx], non_basic_cost[cnMinIndx]
        
#         # switch global index tracker indices
#         cindx[cnMinIndx], cindx[ratioMinIndx + nonbasicSize] = cindx[ratioMinIndx + nonbasicSize], cindx[cnMinIndx]

# print(Simplex(A, b, c))

In [ ]:
#print(sp.optimize.linprog(A_eq=A, b_eq=b, c=c, method='simplex', options={'bland': True}))